# Howto Log Sum Exp

Using flash-attention intensively you will at some point hear about `lse` values being returend. `lse` or log sum exp can be used to compute softmax (and thereby also attention) in a blockwise fashion. This notebook aaims to explain how this works.

Let's start by defining a naive softmax function ..

In [85]:
from typing import Tuple, Sequence
import torch
import math

def naive_softmax(x: torch.Tensor) -> torch.Tensor:
    return x.exp() / x.exp().sum()

... and verify that it's output matches the output of the official `torch.softmax()` function.

In [86]:
x = torch.randn(10)  # generate normally distributed random numbers
a = torch.softmax(x, dim=-1) # reference output
b = naive_softmax(x) # our naive version

print("a", a)
print("b", b)
print("allcclose", torch.allclose(a, b, atol=1e-6))

a tensor([0.0113, 0.0985, 0.1366, 0.1658, 0.2436, 0.0925, 0.0238, 0.0939, 0.0608,
        0.0733])
b tensor([0.0113, 0.0985, 0.1366, 0.1658, 0.2436, 0.0925, 0.0238, 0.0939, 0.0608,
        0.0733])
allcclose True


Our naive softmax function has a problem with numerical stability when it get input vectors with larger elements:

In [87]:
naive_softmax(x * 100)

tensor([0., 0., 0., nan, nan, 0., 0., 0., 0., 0.])

But before we will look into how to fix this let's first look at how a blockwise computation of softmax can be implemented with `naive_softmax()`. We generate a vector and split it into two chunks of equal size and compute softmax of the chunks individually.

In [88]:
x = torch.randn(10)

x1,x2 = torch.chunk(x, 2)
s1 = naive_softmax(x1)
s2 = naive_softmax(x2)

print("We have:")
print(f"s1 = {s1}")
print(f"s2 = {s2}")

target = naive_softmax(x)
print("We want:")
print(f"target = {s}")

We have:
s1 = tensor([0.1411, 0.1636, 0.0206, 0.6230, 0.0518])
s2 = tensor([0.1150, 0.4318, 0.0457, 0.0820, 0.3254])
We want:
target = tensor([0.0586, 0.0118, 0.0048, 0.0119, 0.0144, 0.0147, 0.0370, 0.0435, 0.0064,
        0.0116, 0.0723, 0.0200, 0.0224, 0.1518, 0.3166, 0.0333, 0.0943, 0.0225,
        0.0115, 0.0405])


If we look at `naive_softmax()` we note that its output is divided by `x.exp().sum()`. We can call this the "sum exp" value (note the similarity to "log sum exp") and we can use it to "undo" the softmax normalization and thereby compute combine multiple softmax chunks if we have this vaue for each chunk.

In [89]:
se_x1 = x1.exp().sum()
se_x2 = x2.exp().sum()
s1_corrected = s1 * se_x1 / (se_x1 + se_x2)
s2_corrected = s2 * se_x2 / (se_x1 + se_x2)

print("After correction with help of lse values:")
s_combined = torch.cat([s1_corrected, s2_corrected])
print("s_combined", s_combined)

print("allclose(s_combined, target):", torch.allclose(s_combined, target))


After correction with help of lse values:
s_combined tensor([0.0505, 0.0586, 0.0074, 0.2232, 0.0186, 0.0738, 0.2771, 0.0293, 0.0526,
        0.2088])
allclose(s_combined, target): True


... but is this helpful at all? Yes, and it becomes more obivous when we realize that we can return this value from our softmax function and we can do the correction in a blockwise fashion in a loop by accumulating the lse value:

In [90]:
def naive_softmax2(x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    exp_sum = x.exp().sum()
    return x.exp() / exp_sum, exp_sum


def naive_blockwise_softmax(blocks: Sequence[torch.Tensor]) -> torch.Tensor:
    out = []
    exp_sum = 0
    for block in blocks:
        block_softmax, block_sum_exp = naive_softmax2(block)
        for o in out:
            o *= exp_sum / (exp_sum + block_sum_exp)
        
        out.append(block_softmax * block_sum_exp / (block_sum_exp + exp_sum))
        exp_sum += block_sum_exp
        
    return torch.cat(out)
    

x_long = torch.randn(20)
chunks = torch.chunk(x_long, 4)
print(naive_blockwise_softmax(chunks))
print(torch.softmax(x_long, dim=-1))


tensor([0.0500, 0.0066, 0.0234, 0.0261, 0.0528, 0.0184, 0.0217, 0.1169, 0.0282,
        0.0131, 0.0274, 0.0812, 0.0592, 0.4354, 0.0068, 0.0071, 0.0023, 0.0067,
        0.0113, 0.0054])
tensor([0.0500, 0.0066, 0.0234, 0.0261, 0.0528, 0.0184, 0.0217, 0.1169, 0.0282,
        0.0131, 0.0274, 0.0812, 0.0592, 0.4354, 0.0068, 0.0071, 0.0023, 0.0067,
        0.0113, 0.0054])


OK, then now let's look at the numerical stability of softmax. First we can observe a interesting property of the softmax function: it's output is shift/translation invariant:

In [91]:
x = torch.randn(8)
print(naive_softmax(x))
print(naive_softmax(x+5))
print(naive_softmax(x-3))

tensor([0.2243, 0.1702, 0.1054, 0.1254, 0.0674, 0.1519, 0.0710, 0.0844])
tensor([0.2243, 0.1702, 0.1054, 0.1254, 0.0674, 0.1519, 0.0710, 0.0844])
tensor([0.2243, 0.1702, 0.1054, 0.1254, 0.0674, 0.1519, 0.0710, 0.0844])


This porperty allows us to deal with problematic large inputs simply by subtracting their maximum:

In [92]:
def stable_softmax(x):
    m = x.max()
    return (x-m).exp() / (x-m).exp().sum()

This "stable" function now can also deal with larger value that were problematic for our naive function:

In [93]:
large_input = torch.randn(10) * 100

print("naive: ", naive_softmax(large_input))
print("stable: ", stable_softmax(large_input))
print("torch: ", torch.softmax(large_input, dim=-1))


naive:  tensor([nan, nan, 0., 0., 0., 0., nan, 0., 0., 0.])
stable:  tensor([8.8560e-23, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        8.3795e-27, 0.0000e+00, 0.0000e+00, 0.0000e+00])
torch:  tensor([8.8560e-23, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        8.3795e-27, 0.0000e+00, 0.0000e+00, 0.0000e+00])


In [94]:
def stable_softmax2(x):
    """returns softmax result and log sum exp"""
    m = x.max()
    a = (x - m).exp()
    b = a.sum()
    lse = m + torch.log(b)
    return a / b, lse

Again we can now use this to combine two softmax block results, but to do it in the same way as before we would need to calculate the exp() values.. which is as we know numerically not stable:

In [100]:
x = torch.randn(20)

a = torch.softmax(x, dim=-1)

x1, x2 = x.chunk(2)

b1, lse1 = stable_softmax2(x1)
b2, lse2 = stable_softmax2(x2)

c1 = b1 * torch.exp(lse1) / (torch.exp(lse1) + torch.exp(lse2))
c2 = b2 * torch.exp(lse2) / (torch.exp(lse1) + torch.exp(lse2))

print(a)
print(torch.cat([c1, c2]), torch.allclose(a, torch.cat([c1, c2])))


tensor([0.0629, 0.0096, 0.0210, 0.0297, 0.0161, 0.0350, 0.0121, 0.0678, 0.0051,
        0.0677, 0.0215, 0.0045, 0.0189, 0.1579, 0.0195, 0.1159, 0.0335, 0.0363,
        0.2333, 0.0315])
tensor([0.0629, 0.0096, 0.0210, 0.0297, 0.0161, 0.0350, 0.0121, 0.0678, 0.0051,
        0.0677, 0.0215, 0.0045, 0.0189, 0.1579, 0.0195, 0.1159, 0.0335, 0.0363,
        0.2333, 0.0315]) True


But luckily log & exp are to the rescue:

In [102]:
d1 = b1 * torch.exp(-torch.log(1 + torch.exp(lse2 - lse1)))
d2 = b2 * torch.exp(-torch.log(1 + torch.exp(lse1 - lse2)))
print(a)
print(torch.cat([d1, d2]), torch.allclose(a, torch.cat([d1, d2])))

tensor([0.0629, 0.0096, 0.0210, 0.0297, 0.0161, 0.0350, 0.0121, 0.0678, 0.0051,
        0.0677, 0.0215, 0.0045, 0.0189, 0.1579, 0.0195, 0.1159, 0.0335, 0.0363,
        0.2333, 0.0315])
tensor([0.0629, 0.0096, 0.0210, 0.0297, 0.0161, 0.0350, 0.0121, 0.0678, 0.0051,
        0.0677, 0.0215, 0.0045, 0.0189, 0.1579, 0.0195, 0.1159, 0.0335, 0.0363,
        0.2333, 0.0315]) True


To understand why `b1 * torch.exp(lse1) / (torch.exp(lse1) + torch.exp(lse2))` is equal to `b1 * torch.exp(-torch.log(1 + torch.exp(lse2-lse1)))` we remember school math basics:

In [96]:
a = 5
b = 3

print("math.exp(5)/math.exp(3) =", math.exp(5) / math.exp(3))
print("math.exp(5 - 3) =", math.exp(5 - 3))

print("a/(a+b) =", a / (a+b))
print("1/(1+b/a) =", 1 / (1+b/a))

math.exp(5)/math.exp(3) = 7.38905609893065
math.exp(5 - 3) = 7.38905609893065
a/(a+b) = 0.625
1/(1+b/a) = 0.625


With the fresh knowledge about softmax we can now take a look at the `update()` function that is used in the ring-flash-attention implementation:

In [97]:
def _update_out_and_lse(
    out: torch.Tensor,
    lse: torch.Tensor,
    block_out: torch.Tensor,
    block_lse: torch.Tensor,
) -> Tuple[torch.Tensor, torch.Tensor]:
    block_out = block_out.to(torch.float32)
    block_lse = block_lse.transpose(-2, -1).unsqueeze(dim=-1)

    new_lse = lse + torch.log(1 + torch.exp(block_lse - lse))
    out = torch.exp(lse - new_lse) * out + torch.exp(block_lse - new_lse) * block_out

    lse = new_lse
    return out, lse